### Read txt files

In [22]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
import h5py
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
 

image_base_path = 'nabirds/images/'

In [23]:
images = "nabirds/images.txt"
train_test_split = "nabirds/train_test_split.txt"
classes = "nabirds/classes.txt"
image_class_labels = "nabirds/image_class_labels.txt"

def load_txt():
    img_dic = {}
    train_dic = {}
    class_dic = {}
    image_class_dic = {}
    
    with open(images) as img_object:
        for line in img_object:
            vals = line.replace('\n', '').split(' ')
            img_dic[vals[0]] = vals[1]
            
    with open(train_test_split) as train_object:
        for line in train_object:
            vals = line.replace('\n', '').split(' ')
            train_dic[vals[0]] = vals[1]
            
    with open(classes) as class_object:
        for line in class_object:
            vals = line.replace('\n', '').split(' ')
            key = vals[0]
            vals.remove(key)
            val = ' '.join(vals)
            class_dic[key] = (val)
            
    with open(image_class_labels) as image_class_object:
        for line in image_class_object:
            vals = line.replace('\n', '').split(' ')
            image_class_dic[vals[0]] = vals[1]
    
    image_infos = []
    for key in img_dic:
        location = img_dic[key]
        isTraining = bool(int(train_dic[key]))
        class_name = class_dic[image_class_dic[key]]
        image_infos.append([location, isTraining, class_name])
    return image_infos

def getTestInfo():
    data = [d for d in load_txt() if d[1] == True]
    return data

def getTrainInfo():
    data = [d for d in load_txt() if d[1] == False]
    return data

In [24]:
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(224, 224))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(224,224,3)
    
#     img = img.astype('float32')  
#     img = img - [123.68, 116.779, 103.939]  
    return img

In [25]:
train_info = getTrainInfo()

def load_images(infos):
    images = []
    i = 0
    for info in infos:
        if(i < 100):
            images.append(load_image(image_base_path + info[0]))
        else:
            break
        i = i + 1
    return np.array(images)

targets = [[target[2]] for target in train_info]

targets = [targets[i] for i in range(100)]

images = load_images(train_info)

encoder = OneHotEncoder()
targets = np.array(encoder.fit_transform(targets).toarray())




In [41]:
# define cnn model 
def define_model():  
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(94, activation='softmax'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# run the test harness for evaluating a model
def run_test_harness():
# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
#     featurewise_center=True
	# specify imagenet mean values for centering
# 	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow(images, targets)
	# fit model
	model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=10, verbose=1)
	# save model
	model.save('final_model.h5')
 
# entry point, run the test harness
run_test_harness()

Epoch 1/10
4/4 [==============================] - 11s 2s/step - loss: 5.2042 - accuracy: 0.0227
Epoch 2/10
4/4 [==============================] - 11s 2s/step - loss: 5.1819 - accuracy: 0.0344
Epoch 3/10
4/4 [==============================] - 10s 2s/step - loss: 5.0752 - accuracy: 0.0069
Epoch 4/10
4/4 [==============================] - 11s 2s/step - loss: 4.7728 - accuracy: 0.0000e+00
Epoch 5/10
4/4 [==============================] - 11s 2s/step - loss: 4.4132 - accuracy: 0.0464
Epoch 6/10
4/4 [==============================] - 10s 2s/step - loss: 4.3460 - accuracy: 0.0615
Epoch 7/10
4/4 [==============================] - 10s 3s/step - loss: 4.5989 - accuracy: 0.1134
Epoch 8/10
4/4 [==============================] - 10s 3s/step - loss: 4.5847 - accuracy: 0.0194
Epoch 9/10
4/4 [==============================] - 11s 2s/step - loss: 4.4275 - accuracy: 0.0221
Epoch 10/10
4/4 [==============================] - 10s 2s/step - loss: 4.1489 - accuracy: 0.0257


In [42]:
# make a prediction for a new image.

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(224, 224))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 224, 224, 3)
	# center pixel data
	return img
 
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('nabirds/images/0817/2d9d025b88c34a1ebf9eb88f21f23ad8.jpg')
	# load model
	model = load_model('final_model.h5')
	# predict the class
	result = model.predict(img)
	print(result[0])
 
# entry point, run the example
run_example()

[3.63455115e-07 5.60537633e-03 3.36288540e-05 2.03731470e-04
 4.87527076e-12 2.43435238e-06 9.58965917e-04 4.52843233e-06
 8.60197033e-05 7.00839706e-08 3.57615090e-06 1.13094866e-03
 5.97946167e-01 1.47321924e-08 3.46806650e-09 6.50239934e-04
 1.12975407e-07 4.39693504e-05 2.44007694e-08 3.65483034e-12
 5.15235661e-05 2.86936793e-05 3.73611721e-04 6.40326820e-04
 9.39968381e-11 8.16277979e-09 1.17346675e-08 8.53693753e-04
 2.01977397e-04 1.79359540e-05 4.98255424e-04 3.54966323e-05
 6.57565380e-08 4.50885971e-04 1.33444793e-08 1.63314233e-08
 3.06000441e-01 7.00452673e-12 3.54027812e-04 6.49369849e-06
 3.47381790e-10 2.36073261e-04 4.36309406e-08 3.08158263e-08
 1.27707463e-05 7.76223260e-07 1.77521076e-06 3.73857262e-07
 1.29590346e-03 4.77890061e-10 1.43615397e-10 1.90558946e-08
 2.73998652e-04 5.33252287e-09 8.15668027e-05 1.51941558e-05
 8.65102550e-07 1.26020021e-08 1.43042498e-03 1.78597759e-07
 4.34419833e-10 2.55747636e-06 4.36153128e-11 7.61959609e-06
 3.34938022e-06 2.700145